# Intro to Volatility Forecasting (With the S&P500!)


### Contents
1. [Motivation](#chapter1)
2. [Variance at Risk (VaR)](#chapter2)
3. [Expected Shortfall (ES)](#chapter3)
4. [Univariate Volatility Models](#chapter4)  
   4.1 [Moving Average (MA)](#subsec1)  
   4.2 [Exponentially Weighted Moving Average (EWMA)](#subsec2)  
   4.3 [Autoregressive Conditional Heteroskedasticity (ARCH)](#subsec3)  
   4.4 [Generalized Autoregressive Conditional Heteroskedasticity (GARCH)](#subsec4)

## 1. Motivation <a class = "anchor" id = "chapter1"></a>

When accounting for random processes, it is often where just the point estimate of the forecasted values are predicted. However, knowing the potential distribution around that estimate could provide crucial information such as probability of a loss in a portfolio, as will be examined in this notebook. As an example I will be going over forecasting variance for an S&P500 etf using closing data from 2020-01-01 to 2024-11-30.

In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("/kaggle/input/s-and-p500-etf-data-2020-01-01-to-2024-11-30/spy-data.csv")
df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
df["SPY"] = df["SPY"].pct_change()
df["SPY"] = df["SPY"].apply(lambda x: x*100)
df.dropna(inplace=True)

In [3]:
df.head()

,Date,SPY
1,2020-01-03,-0.757212
2,2020-01-06,0.381480
3,2020-01-07,-0.281158
4,2020-01-08,0.532938
5,2020-01-09,0.678081


## 2. Variance at Risk (VaR) <a class = "anchor" id = "chapter2"></a>  

**Value at Risk (VaR):** The value at risk is at what quantity will your portfolio be in the $\alpha$ percentile.  
$r_t^{VaR(\alpha)}$ is the value at which $P(r_t <= r_t^{VaR(\alpha)} = \alpha)$
  
Eg. We wish to see how much a portfolio of \\$10,000 in SPY will return tomorrow. But we also want to make sure that our portfolio might not lose all our money. Therefore, just to be safe, we evaluate the value the portfolio has a 95\% chance of returning higher than tomorrow. This essentially says, what is the value at risk of a portfolio invested SPY given $\alpha = 0.05$

In [4]:
# more imports
from scipy.stats import norm

In [5]:
def var(sigma, mu, alpha):
    # assuming normal distribution here for easier computations
    z_value = norm.ppf(alpha)
    return mu + z_value * sigma

This implementation is a very simple calculation of Value at Risk using the standard deviation of the variable in the past data as sigma and using the average of variable in the past data as mu. In actuality, the implementation will use some sort of mean and variance model (see below).

In [6]:
# implementing with the example
alpha = 0.05

std = df["SPY"].std()
mean = df["SPY"].mean()

value_at_risk_1_day = var(std, mean, alpha)
value_at_risk_1_day

-2.120907987267147

So this means that our portfolio invested entirely in the SPY etf will have a 95% chance of returning more than -2.12% tomrrow using this very simple model.

## 3. Expected Shortfall <a class = "anchor" id = "chapter3"></a>

**Expected Shortfall:** On average how much can we expect to lose beyond the value at risk.   
Mathematicall it is:  
$$ES(\alpha) = E[r_t|r_t < r_t^{VaR(\alpha)}]$$
For a the standard normally distributed random variable $z_t$
$$E[z_t|z_t < z_a] = \frac{1}{\alpha \sqrt{2\pi}}exp(-z^2_{\alpha}/2)$$
Since $$z_t  = \frac{r_t - \mu_{t|t-1}}{\sigma_{t|t-1}}$$
We get
$$r_t^{ES(\alpha)} = \mu_{t|t-1}+z_t * \sigma_{t|t-1}$$

The value $\mu$ is forecasted using a mean model and the value for $\sigma$ is forecasted using a volatility model.  
  
Alone, value at risk only tells the upper bound for how much we might lose but not the actual amount we might lose if we somehow found ourselves in a situation significantly underperforming. The expected shortfall is essentially an average for a worst case scenario of what we might lose.  

Following our example, if we wanted to investigate the Expected Shortfall, we will be asking the question: if the S&P500 ETF SPY performed worse than it does 95% of the time, what loss can we expect?

In [7]:
import numpy as np

In [8]:
# assuming differences in price data is normally distributed
def expected_shortfall(alpha, mu, sigma):
    z_a = norm.ppf(alpha)
    z_t = (1 / (alpha * np.sqrt(2 * np.pi))) * np.exp(- z_a**2 / 2)
    return mu - z_t * sigma  # note: z_t is negative since its the upper bound of the lower tail in the standard normal distribution

In [9]:
# Following the simple example given in the VaR section, this implementation of ES will follow using the same numbers.
alpha = 0.05

std = df['SPY'].std()
mean = df['SPY'].mean()

ES = expected_shortfall(alpha, mean, std)
ES

-2.676144046815667

On average, if the S&P500 ETF SPY were to underperform 95% of it's usual returns, we would expect to lose -2.68%.

## 4. Volatility Models <a class = "anchor" id = "chapter4"></a>

### 4.1 Rolling Window Volatility <a class = "anchor" id = "subsec1"></a>

This is the simplest form of volatility forecasting. Essentially it just takes the average variance for each time t that is given in the "window" of size n. The Rolling Window Volatility model is the volatility equivalent of the Moving Average Smoothing model for mean forecasting aka Simple Moving Average.  

The formula for Rolling Window Volatility is given by:
$$\hat{\sigma}^2_{t|t-1} = \frac{1}{n}\Sigma_{i=1}^{n}(r_{t-i}-\mu)^2$$

Generally higher values of n will give the model more "memory" allowing it to have a smoother prediction as there are more values to average over. This model is rarely used but serves as a baseline for other models to built off of.

In [10]:
# implementation
# note: mu could just be calculated using the data in the function but usually is calculated using some sort of mean
# forecasting function such as ARMA
def ma_vol(n, mu, data):
    roll_var = data['SPY'].iloc[-n:]
    roll_var.apply(lambda x: (x - mu) ** 2)
    return roll_var.sum() / n

In [11]:
ma_vol(10, df["SPY"].mean(), df)

0.15546348489188122

The estimated volatility for the returns on SPY for the next trading day is 0.15%.

### 4.2 Exponentially Weighted Moving Average (EWMA) <a class = "anchor" id = "subsec2"></a>

The Exponentially Weighted Moving Average model  (EWMA) is the volatility equivalent for the Simple Expoential Smoothing model (SES) for mean forecasting.  
SES:
$$l_t = \alpha\Sigma^{T-1}_{k=0}(1-\alpha)^k y_{T-k} + (1-\alpha)^T l_0$$
EWMA:
$$\hat{\sigma}^2_{t|t-1} = (1-\lambda) \Sigma^{t-1}_{i=1}\lambda^{i-1}(r_{t-1}-\mu)^2$$
  
The EWMA model is the SES model where $l_0 = 0$, $\alpha = (1-lambda)$, $k = i - 1$, $T = t - 1$, $y_t = (r_t - \mu)^2$, and $l_{t-1} = \hat{\sigma}^2_{t|t-1}$
Note: The different notation is because one comes from the field of statistics and the other from the field of finance.

In [12]:
# implementation
def EWMA(_lambda, mean, data):
    sigma_squared = 0
    for i in range(1, len(data)):
        r_t_1 = data.iloc[-i]
        sigma_squared += _lambda ** (i - 1) * (r_t_1 - mean) ** 2
    sigma_squared *= (1 - _lambda)
    return sigma_squared

In [13]:
EWMA(0.3, df["SPY"].mean(), df["SPY"])

0.26054840450862227

Since the EWMA model takes in all the data rather than just the most recent n values as the Rolling Window Model did, the EWMA model is described to have a longer memory, making it generally more reliable. However, it is still not used often in forecasting as it is often a part of more complicated and sophisticated models such as GARCH.

### 4.3 Autoregressive Conditional Heteroskedasticity (ARCH) <a class = "anchor" id = "subsec3"></a>
  
The Autoregressive Conditional Heteroskedasticity model of order p (ARCH(p)) is defined as:
$$\sigma^2_{t|t-1} = \omega + \alpha_1 \epsilon^2_{t-1} + \alpha_2 \epsilon^2_{t-2} + ... + \alpha_p \epsilon^2_{t-p}$$
Where $\omega, \alpha_i$ are parameters fit using data and $\epsilon_i$ is the error term of the mean forecast for the time i.  
Because $\sigma^2_{t|t-1} >= 0$ then $\omega >= 0$  
  
Notice how the ARCH model of order p only has p terms of $\epsilon$. This once again means that the ARCH model only has a short memory and thus may not be very appropriate to be used in practice. In order to have a sufficiently precise estimate with the ARCH model, p has to be significantly large. Another key feature about the ARCH model is that it performs when when variance is clustered. This is due to the small amount of epsilons being able to capture time specific trends which can result in clustering of variances.

In [14]:
%pip install arch --quiet
from arch import arch_model

Note: you may need to restart the kernel to use updated packages.


In [15]:
# implementation
p = 1
model = arch_model(df['SPY'], vol='ARCH', p=p)

fitted_model_arch_1 = model.fit(disp='off')

In [16]:
fitted_model_arch_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Constant Mean - ARCH Model Results                      
==============================================================================
Dep. Variable:                    SPY   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -1939.90
Distribution:                  Normal   AIC:                           3885.81
Method:            Maximum Likelihood   BIC:                           3901.17
                                        No. Observations:                 1236
Date:                Sun, Dec 01 2024   Df Residuals:                     1235
Time:                        15:56:21   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.1472  3.348e-02      4.398  1.093e-05 [8.162e-02,  0.213]
                            Volatility Model                            
========================================================================
                 coef    std err          t      P>|t|  95.0% Conf. Int.
------------------------------------------------------------------------
omega          0.8863  8.414e-02     10.535  5.989e-26 [  0.721,  1.051]
alpha[1]       0.5249      0.123      4.250  2.134e-05 [  0.283,  0.767]
========================================================================

Covariance estimator: robust
"""

In [17]:
# with a larger p
p = 20
model = arch_model(df['SPY'], vol='ARCH', p=p)

fitted_model_arch_2 = model.fit(disp='off')

In [18]:
fitted_model_arch_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                      Constant Mean - ARCH Model Results                      
==============================================================================
Dep. Variable:                    SPY   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -1781.84
Distribution:                  Normal   AIC:                           3607.68
Method:            Maximum Likelihood   BIC:                           3720.31
                                        No. Observations:                 1236
Date:                Sun, Dec 01 2024   Df Residuals:                     1235
Time:                        15:56:22   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0943  2.702e-02      3.491  4.814e-04 [4.137e-02,  0.147]
                               Volatility Model                              
=============================================================================
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.2039  6.200e-02      3.288  1.009e-03    [8.234e-02,  0.325]
alpha[1]       0.0998  5.595e-02      1.784  7.450e-02   [-9.873e-03,  0.209]
alpha[2]       0.1672  4.016e-02      4.162  3.151e-05    [8.845e-02,  0.246]
alpha[3]       0.0962  4.238e-02      2.271  2.314e-02    [1.319e-02,  0.179]
alpha[4]       0.0984  5.490e-02      1.792  7.310e-02   [-9.209e-03,  0.206]
alpha[5]       0.0650  4.618e-02      1.408      0.159   [-2.550e-02,  0.156]
alpha[6]       0.0665  4.180e-02      1.591      0.112   [-1.543e-02,  0.148]
alpha[7]       0.0000  4.501e-02      0.000      1.000 [-8.821e-02,8.821e-02]
alpha[8]       0.0385  3.788e-02      1.016      0.310   [-3.575e-02,  0.113]
alpha[9]       0.0106  2.194e-02      0.484      0.628 [-3.237e-02,5.362e-02]
alpha[10]      0.0000  5.535e-02      0.000      1.000      [ -0.108,  0.108]
alpha[11]      0.0302  2.975e-02      1.016      0.310 [-2.809e-02,8.852e-02]
alpha[12]      0.0000  3.609e-02      0.000      1.000 [-7.073e-02,7.073e-02]
alpha[13]      0.0517  3.145e-02      1.643      0.100   [-9.970e-03,  0.113]
alpha[14]      0.0000  4.413e-02      0.000      1.000 [-8.649e-02,8.649e-02]
alpha[15]  2.5214e-11  3.675e-02  6.861e-10      1.000 [-7.203e-02,7.203e-02]
alpha[16]      0.0559  4.194e-02      1.334      0.182   [-2.626e-02,  0.138]
alpha[17]  4.9449e-12  3.715e-02  1.331e-10      1.000 [-7.281e-02,7.281e-02]
alpha[18]      0.0000  4.515e-02      0.000      1.000 [-8.850e-02,8.850e-02]
alpha[19]  7.7241e-03  2.907e-02      0.266      0.790 [-4.926e-02,6.471e-02]
alpha[20]      0.0726  4.328e-02      1.679  9.325e-02   [-1.218e-02,  0.157]
=============================================================================

Covariance estimator: robust
"""

### 4.4 Generalized Autoregressive Conditional Heteroskedasticity (GARCH) <a class = "anchor" id = "subsec4"></a>
The Generalized ARCH model of order p, q (GARCH(p, q)) is defined as:
$$\sigma^2_{t|t-1} = \omega + \alpha_1 \epsilon^2_{t-1} + \alpha_2 \epsilon^2_{t-2} + ... + \alpha_p \epsilon^2_{t-p} + \beta \sigma^2_{t-1|t-2} + \beta_2 \sigma^2 _{t-2|t-3} + ... + \beta_q \sigma^2_{t-1|t-q-1}$$
The GARCH model adds addition terms to the arch model in the form of conditional variances. Adding these conditional variances allows the model to take data from further in the past into considering, giving it a high degree of accuracy. Generally GARCH(p, q) only need a small amount of paramters to make accurate predictions.

Consider the GARCH(1, 1) model:
$$\sigma^2_{t|t-1} = \omega + \alpha \epsilon^2_{t-1} + \beta \sigma^2_{t-1|t-2}$$
After Exanpanding and substituting
$$\sigma^2_{t|t-1} = \frac{\omega}{1-\beta} + \alpha \Sigma^\infty_{i=1} \beta^{i-1}\epsilon^2_{t-i}$$
Making the GARCH(1, 1) model equivalent to the ARCH($\infty$) model but with significantly less parameters.

In [19]:
# implementation
model = arch_model(df['SPY'], vol='Garch', p=1, q=1)
fitted_model_garch = model.fit(disp='off')

In [20]:
fitted_model_garch.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                     Constant Mean - GARCH Model Results                      
==============================================================================
Dep. Variable:                    SPY   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -1794.80
Distribution:                  Normal   AIC:                           3597.61
Method:            Maximum Likelihood   BIC:                           3618.08
                                        No. Observations:                 1236
Date:                Sun, Dec 01 2024   Df Residuals:                     1235
Time:                        15:56:22   Df Model:                            1
                                Mean Model                                
==========================================================================
                 coef    std err          t      P>|t|    95.0% Conf. Int.
--------------------------------------------------------------------------
mu             0.0961  2.547e-02      3.774  1.604e-04 [4.620e-02,  0.146]
                              Volatility Model                              
============================================================================
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0477  1.798e-02      2.654  7.953e-03 [1.248e-02,8.296e-02]
alpha[1]       0.1568  3.542e-02      4.426  9.601e-06   [8.735e-02,  0.226]
beta[1]        0.8113  3.998e-02     20.290  1.566e-91     [  0.733,  0.890]
============================================================================

Covariance estimator: robust
"""